# install dependancies, takes around 45 seconds

Rendering Dependancies



In [32]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay tensorflow==2.3.1 keras-rl2 > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg piglet > /dev/null 2>&1

Pacman Dependancies

In [33]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools > /dev/null 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

# Imports and Helper functions


In [34]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [35]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [36]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [6]:
#get roms
!wget http://www.atarimania.com/roms/Roms.rar > /dev/null 2>&1
!unrar e Roms.rar > /dev/null 2>&1
!unzip ROMS.zip > /dev/null 2>&1

^C


In [7]:
!python -m atari_py.import_roms ROMS > /dev/null 2>&1

# Pacman!

In [42]:
env = wrap_env(gym.make("MsPacman-v0"))

In [43]:
#check out the pacman action space!
print(env.action_space)

Discrete(9)


In [44]:
env.unwrapped.get_action_meanings()

['NOOP',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'DOWNRIGHT',
 'DOWNLEFT']

In [45]:
actions = env.action_space.n

In [46]:
print(env.observation_space)
height, width, channels = env.observation_space.shape

Box(0, 255, (210, 160, 3), uint8)


In [47]:
episodes = 5

for episode in range(episodes):
  state = env.reset()
  done = False
  score = 0

  while not done:
    env.render(mode='rgb_array')
    action = random.choice([0,1,2,3,4,5,6,7,8])
    n_state, reward, done, info = env.step(action)
    score += reward
  print('Episode:', episode, ' Score:', score)
env.close()
show_video()

Episode: 0  Score: 260.0
Episode: 1  Score: 250.0
Episode: 2  Score: 270.0
Episode: 3  Score: 210.0
Episode: 4  Score: 280.0


**Modeling**

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, SpatialDropout2D
from tensorflow.keras.optimizers import Adam

In [52]:
def build_model(height, width, channels, actions):
  model = Sequential([
                      Conv2D(32, (8, 8), strides=(4,4), activation = 'relu', input_shape = (3, height, width, channels)),
                      Conv2D(64, (4, 4), strides=(2,2), activation = 'relu'),
                      Conv2D(64, (3, 3), activation = 'relu'),
                      Flatten(),
                      Dense(256, activation = 'relu'),
                      Dropout(0.2),
                      Dense(128, activation = 'relu'),
                      Dropout(0.2),
                      Dense(actions, activation = 'linear')
                    ])
  return model

In [58]:
del model

In [59]:
model = build_model(height, width, channels, actions)

In [54]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               17301760  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

In [55]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [60]:
def build_agent(model, actions):
  policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),
                                attr = 'eps',
                                value_max = 1.,
                                value_min = .1,
                                value_test = .2,
                                nb_steps = 10000
                               )
  memory = SequentialMemory(limit = 1000, window_length = 3)
  dqn = DQNAgent(model=model,
                 memory = memory,
                 policy = policy,
                 enable_dueling_network = True,
                 dueling_type = 'avg',
                 nb_actions = actions,
                 nb_steps_warmup = 1000)
  return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate = 0.0001))
dqn.fit(env, nb_steps = 10000, visualize = False, verbose = 2)

Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  589/10000: episode: 1, duration: 8.142s, episode steps: 589, steps per second:  72, episode reward: 130.000, mean reward:  0.221 [ 0.000, 10.000], mean action: 4.234 [0.000, 8.000],  loss: --, mean_q: --, mean_eps: --
 1244/10000: episode: 2, duration: 174.883s, episode steps: 655, steps per second:   4, episode reward: 150.000, mean reward:  0.229 [ 0.000, 10.000], mean action: 4.203 [0.000, 8.000],  loss: 489.209672, mean_q: 108.664345, mean_eps: 0.899020
 1904/10000: episode: 3, duration: 452.936s, episode steps: 660, steps per second:   1, episode reward: 220.000, mean reward:  0.333 [ 0.000, 10.000], mean action: 4.032 [0.000, 8.000],  loss: 145.388591, mean_q: 99.745089, mean_eps: 0.858385
 2544/10000: episode: 4, duration: 439.302s, episode steps: 640, steps per second:   1, episode reward: 250.000, mean reward:  0.391 [ 0.000, 10.00

In [1]:
scores = dqn.test(env, nb_episodes = 10, visualize = True)
print(np.mean(scores.history['episode_reward']))

NameError: ignored

In [ ]:
dqn.save_weights('space_invaders/dqn_weights.h5f')

In [ ]:
observation = env.reset()

while True:
  
    env.render()
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action) 
        
    if done: 
      break;
            
env.close()
show_video()